<a href="https://colab.research.google.com/github/emanbuc/LLM-playground/blob/master/TestAutomazioneMassime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test Estrazione Massima

In [16]:
!pip install OpenAI

In [17]:
!pip freeze > requirements.txt

Per caricare API KEY da variabili di ambiente



```
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
```



In [18]:
#carica API KEY da secret di Google Colab
from google.colab import userdata
openai_key= userdata.get('openai-key-nomos-ai')

In [19]:
from openai import OpenAI
client = OpenAI(api_key=openai_key)

In [20]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [21]:
def get_completion_and_token_count(messages,
                                   model="gpt-3.5-turbo", #model="gpt-4-turbo-preview", #model="gpt-3.5-turbo",
                                   temperature=0,
                                   max_tokens=500):

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message.content

    token_dict = {
'prompt_tokens':response.usage.prompt_tokens,
'completion_tokens':response.usage.completion_tokens,
'total_tokens':response.usage.total_tokens,
    }

    return content, token_dict

In [22]:
config= {
    "massime_path": "/gdrive/MyDrive/NomosAI/Massime",
    "sentenze_path":"/gdrive/MyDrive/NomosAI/Sentenze",
    "prompt_path":"/gdrive/MyDrive/NomosAI/Prompt",
    "models":["gpt-4-turbo","gpt-4o","gpt-3.5-turbo"],
    "prompts":["system_message_C.txt","system_message_B.txt"]
}

In [23]:
config["massime_path"]

'/gdrive/MyDrive/NomosAI/Massime'

In [24]:
def create_text_file_dict_from_folder(folder_path):
    import os
    import glob
    # Initialize an empty dictionary
    text_files_dict = {}

    # Create the pattern for .txt files
    pattern = os.path.join(folder_path, '*.txt')

    # Find all .txt files matching the pattern
    for file_path in glob.glob(pattern):
        # Extract the file name from the file path
        file_name = os.path.basename(file_path)

        # Read the content of the file
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Add the file name and content to the dictionary
        text_files_dict[file_name] = content

    return text_files_dict

In [25]:
prompts_files_dict = create_text_file_dict_from_folder(config["prompt_path"])
for file_name, content in prompts_files_dict.items():
    print(f'File: {file_name}\n')

File: system_message_A.txt

File: system_message_B.txt

File: system_message_C.txt



In [26]:
def save_output_as_text_file(filename,text_content):
  percorso_file = os.path.join(config["massime_path"], filename)
  with open(percorso_file+'.txt', 'w') as f:
    f.write(text_content)

In [29]:
def genera_massima(prompt_file,sentenza,model,output_file_name):

    prompt = prompts_files_dict[prompt_file]
    messages = [
      {'role':'system',
      'content':prompt},
      {'role':'user',
      'content':sentenza},
      ]

    response, token_dict = get_completion_and_token_count(messages,model=model,max_tokens= 1000)
    #print(response)
    print(token_dict)
    save_output_as_text_file(output_file_name+"_"+model+"_"+prompt_file,response)

In [30]:
import os
for file in os.listdir(config["sentenze_path"]):
    # Verifica se il file è un file di testo
    if file.endswith('.txt'):
        # Costruisci il percorso completo al file
        percorso_file = os.path.join(config["sentenze_path"], file)
        # Apri il file e leggi il contenuto
        with open(percorso_file, 'r') as f:
            sentenza = f.read()

            for prompt_file in config["prompts"]:
              for model in config["models"]:
                genera_massima(prompt_file,sentenza,model,file)


{'prompt_tokens': 1457, 'completion_tokens': 31, 'total_tokens': 1488}
{'prompt_tokens': 1242, 'completion_tokens': 74, 'total_tokens': 1316}
{'prompt_tokens': 1457, 'completion_tokens': 18, 'total_tokens': 1475}
{'prompt_tokens': 1336, 'completion_tokens': 68, 'total_tokens': 1404}
{'prompt_tokens': 1144, 'completion_tokens': 36, 'total_tokens': 1180}
{'prompt_tokens': 1336, 'completion_tokens': 43, 'total_tokens': 1379}


KeyboardInterrupt: 